In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from gensim.models import word2vec
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize 
%pip install contractions
import contractions
import string
from gensim.parsing.preprocessing import STOPWORDS
from collections import Counter
%pip install imbalanced-learn
from imblearn.under_sampling import RandomUnderSampler

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
     |████████████████████████████████| 321 kB 15.3 MB/s 
     |████████████████████████████████| 284 kB 54.2 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85437 sha256=9e922ba20afd0ce46f4fa5507a1cf880046986534ce1acde3018f89b33575f42
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


Importing a combined version of the dataset with 10k rows from each .tsv

In [3]:
df = pd.read_csv('/content/drive/MyDrive/ALDA_Project/amazon_reviews_us_Electronics_v1_00.tsv',sep='\t',error_bad_lines=False )

b'Skipping line 9076: expected 15 fields, saw 22\nSkipping line 19256: expected 15 fields, saw 22\nSkipping line 24313: expected 15 fields, saw 22\nSkipping line 47211: expected 15 fields, saw 22\nSkipping line 54295: expected 15 fields, saw 22\nSkipping line 56641: expected 15 fields, saw 22\nSkipping line 63067: expected 15 fields, saw 22\n'
b'Skipping line 93796: expected 15 fields, saw 22\n'
b'Skipping line 132806: expected 15 fields, saw 22\nSkipping line 164631: expected 15 fields, saw 22\nSkipping line 167019: expected 15 fields, saw 22\nSkipping line 167212: expected 15 fields, saw 22\n'
b'Skipping line 198103: expected 15 fields, saw 22\nSkipping line 199191: expected 15 fields, saw 22\nSkipping line 202841: expected 15 fields, saw 22\nSkipping line 218228: expected 15 fields, saw 22\nSkipping line 235900: expected 15 fields, saw 22\n'
b'Skipping line 277761: expected 15 fields, saw 22\nSkipping line 304582: expected 15 fields, saw 22\nSkipping line 312029: expected 15 fields,

Removing all reviews with a neutral rating


In [4]:
df = df[df.star_rating != 3]
df.shape

(2852637, 15)

In [5]:
df.loc[df['star_rating'] > 3, 'star_rating_sentiment'] = 'positive'

df['star_rating_sentiment'] = df['star_rating_sentiment'].fillna('negative')


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [6]:
reviews = df["review_body"].astype('str').values
labels = df["star_rating_sentiment"].values

In [7]:
rus = RandomUnderSampler()
reviews = reviews.reshape(-1, 1)
reviews, labels = rus.fit_resample(reviews, labels)

In [17]:
len(reviews)

1073698

In [8]:
all_stopwords = STOPWORDS.union(['it','at','me','br','if','to','do'])
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):

    tag = word[0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def process_review(sentence):

  # convert the sentence to lower case, fix contrations and the otkenize
  sentence = word_tokenize(contractions.fix(sentence.lower()))

  # Removing all short words, punctuations and digits
  sentence = [word for word in sentence if len(word)>1 and all(punc not in word for punc in string.punctuation) and not any(char.isdigit() for char in word)]

  # remove all stop words
  sentence = [word for word in sentence if not word in all_stopwords]

  # add a parts of speech tag to each word
  sentence = nltk.pos_tag(sentence)

  # lemmatize each word using itself and its POS
  sentence = [lemmatizer.lemmatize(word[0], get_wordnet_pos(word[1])) for word in sentence]

  return sentence

In [9]:
processed_reviews = []
for review in reviews:
  processed_reviews.append(process_review(review[0]))


In [10]:
word_count = Counter()
sentences_in_words = []
for i in range(len(processed_reviews)):
    sentences_in_words.append(processed_reviews[i])
    
    for word in sentences_in_words[i]:
        
        if len(word)>1 and type(word)==str and string.punctuation not in word:
          word_count.update({str(word): 1})

In [11]:
model = word2vec.Word2Vec(sentences_in_words)

In [14]:
for word_pair in [['good','bad'],['best','worst'],['positive','negative'],['awesome','terrible']]:
  result = []
  correct_count = 0
  for i in range(len(processed_reviews)):
    sum_g = 0
    sum_b = 0
    n=0
    for word in processed_reviews[i]:
      try:
        sum_g += model.wv.similarity(word, word_pair[0])
        sum_b += model.wv.similarity(word, word_pair[1])
      except:
        n-=1
    if sum_g>0:
      avg_g = sum_g/(len(processed_reviews[i])+n)
    else:
      avg_g =0
    if sum_b>0:
      avg_b = sum_b/(len(processed_reviews[i])+n)
    else:
      avg_b =0
    if (avg_g>avg_b and labels[i]=='positive') or (avg_g<=avg_b and labels[i]=='negative'):
      correct_count+=1
    # print(labels[i],avg_g,avg_b,avg_g>avg_b)
  print(word_pair,correct_count/len(processed_reviews))

['good', 'bad'] 0.7311376197031195
['best', 'worst'] 0.6482791250426098
['positive', 'negative'] 0.6003028784630315
['awesome', 'terrible'] 0.7093149097791


In [16]:
for word_pair in [['good','bad'],['best','worst'],['positive','negative'],['awesome','terrible']]:
  result = []
  correct_count = 0
  for i in range(len(processed_reviews)):
    sum=0
    for word in processed_reviews[i]:
      try:
        temp = (1 if model.wv.similarity(word, word_pair[0])>model.wv.similarity(word, word_pair[1]) else -1)
        sum+= temp
      except:
        pass
      
    if (sum>0 and labels[i]=='positive') or (sum<=0 and labels[i]=='negative'):
      correct_count+=1
    # print(labels[i],avg_g,avg_b,avg_g>avg_b)
  print(word_pair,correct_count/len(processed_reviews))

['good', 'bad'] 0.6896725149902486
['best', 'worst'] 0.6174818244981364
['positive', 'negative'] 0.5987614766908386
['awesome', 'terrible'] 0.6637406421544978


*   Good/bad = 0.694
*   best/worst = 54.99%

In [ ]:
for j in range(50,150,5):
  result = []
  correct_count = 0
  for i in range(len(processed_reviews)):
    sum_g = 0
    sum_b = 0
    n=0
    for word in processed_reviews[i]:
      try:
        sum_g += model.wv.similarity(word, 'good')
        sum_b += model.wv.similarity(word, 'bad')
      except:
        n-=1
    if sum_g>0:
      avg_g = sum_g/(len(processed_reviews[i])+n)
    else:
      avg_g =0
    if sum_b>0:
      avg_b = sum_b/(len(processed_reviews[i])+n)
    else:
      avg_b = 0
    if (avg_b==0 and labels[i]=='positive'):
      correct_count+=1
    elif avg_b!=0:
      if (avg_g/avg_b>j/100 and labels[i]=='positive') or (avg_g/avg_b<=j/100 and labels[i]=='negative'):
        correct_count+=1

  print('Threshold: ',j/100 ,' - Accuracy:',correct_count/len(processed_reviews))

Threshold:  0.5  - Accuracy: 0.6471652177800462
Threshold:  0.55  - Accuracy: 0.6564583337214003
Threshold:  0.6  - Accuracy: 0.6657859100044892
Threshold:  0.65  - Accuracy: 0.6750222129500102
Threshold:  0.7  - Accuracy: 0.6841569975915015
Threshold:  0.75  - Accuracy: 0.6931055101155074
Threshold:  0.8  - Accuracy: 0.7018342215408803
Threshold:  0.85  - Accuracy: 0.7098793142950811
Threshold:  0.9  - Accuracy: 0.7173143658645168
Threshold:  0.95  - Accuracy: 0.7242530022408535
Threshold:  1.0  - Accuracy: 0.7302947383714974
Threshold:  1.05  - Accuracy: 0.7356463363068573
Threshold:  1.1  - Accuracy: 0.7400218683465928
Threshold:  1.15  - Accuracy: 0.7431158482180278
Threshold:  1.2  - Accuracy: 0.744866806122392
Threshold:  1.25  - Accuracy: 0.7458903714079751
Threshold:  1.3  - Accuracy: 0.7444299980068884
Threshold:  1.35  - Accuracy: 0.7428904589558702
Threshold:  1.4  - Accuracy: 0.740464264625621
Threshold:  1.45  - Accuracy: 0.7365441679131376
